In [2]:
#这段代码实现的是对苹果颜色的识别并且标记坐标，注意，还不是最理想的模型
#我想要的最后还没有导出为一个文件夹
#而且坐标也不是完全正确，我现在还需要调整阈值，然后让更多人红色能够被识别，同时红色之间还要能够被区分
#我可能还需要从网上去找一些源代码
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取图片
def ReadImg(image_path):
    img = cv2.imread(image_path, 1)
    return img

# 高斯滤波
def GausBlur(src):
    dst = cv2.GaussianBlur(src, (5, 5), 1.5)
    return dst

# 开运算（替代 open_mor）
def open_mor(img):
    kernel = np.ones((5, 5), np.uint8)
    result = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return result

# 红色对象检测
def detect_red_objects(src):
    # 设定红色阈值范围
    lower_red = np.array([0, 0, 100])
    upper_red = np.array([100, 100, 255])

    # 根据阈值构建掩模
    mask = cv2.inRange(src, lower_red, upper_red)

    # 使用掩模提取红色对象
    red_objects = cv2.bitwise_and(src, src, mask=mask)

    return red_objects

# 轮廓拟合
def draw_shape(open_img, src):
    contours, hierarchy = cv2.findContours(open_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    apple_count = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 50:  # 假设红色对象的最小面积为100
            # 获取圆心和半径
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)

            # 绘制圆形
            cv2.circle(src, center, radius, (0, 0, 255), 2)

            # 绘制坐标
            cv2.putText(src, f'({center[0]}, {center[1]})', (center[0] - 30, center[1] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            apple_count += 1
            #打印坐标
            print('苹果坐标:', x, y)
    return src, apple_count

# 处理所有图像
for i in range(1, 201):  # 假设图像编号从1到200
    image_path = f'Attachment_2/{i}_processed.jpg'
    src = ReadImg(image_path)
    gaus_img = GausBlur(src)
    red_objects_img = detect_red_objects(gaus_img)
    gray_img = cv2.cvtColor(red_objects_img, cv2.COLOR_BGR2GRAY)
    _, thres_img = cv2.threshold(gray_img, 1, 255, cv2.THRESH_BINARY)
    open_img = open_mor(thres_img)
    result_img, apple_count = draw_shape(open_img, src)

    # 显示图像
    plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
    plt.title(f'Processed Image {i}, Red Objects Count: {apple_count}')
    plt.show()

SyntaxError: invalid syntax (3110994266.py, line 63)